In [1]:
import sys, os
# sys.path.insert(0, os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
import open3d as o3d
import time
import numpy as np

# Load the Optitrack CSV file parser module.
import optitrack.csv_reader as csv
from optitrack.geometry import *

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation


filename = "../material/60fps/ProvaRigidBody.csv"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# df = csv.CSVReader('../material/60fps/ProvaRigidBody.csv')
# take = csv.Take().readCSV('../material/60fps/ProvaRigidBody.csv')
# print(take.rigid_bodies.keys())  # Display the first few rows to verify


In [3]:
# Read the file.
take = csv.Take().readCSV(filename)

# Print out some statistics
print("Found rigid bodies:", take.rigid_bodies.keys())

# Process the first rigid body into a set of planes.
bodies = take.rigid_bodies

# for now:
xaxis = [1,0,0]
yaxis = [0,1,0]

body_edges = [[0,1],[1,2],[2,3],[3,4],[3,5],[5,6],[6,7],[7,8],[3,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],
                [0,16],[16,17],[17,18],[18,20],[15,19]]

bones_pos = []
if len(bodies) > 0:
    for body in bodies: 
        bones = take.rigid_bodies[body]
        bones_pos.append(bones.positions)
        # for pos,rot in zip(body.positions, body.rotations):
        #     if pos is not None and rot is not None:
        #         xaxis, yaxis = quaternion_to_xaxis_yaxis(rot)
                # plane = rs.PlaneFromFrame(pos, xaxis, yaxis)

                # # create a visible plane, assuming units are in meters
                # rs.AddPlaneSurface( plane, 0.1, 0.1 )
# bones_pos = np.array(bones_pos).T.tolist()
# bones_pos = np.array(bones_pos, dtype=float).T.tolist()
colors = [[1, 0, 0] for i in range(len(body_edges))]
# keypoints = o3d.geometry.PointCloud()
# keypoints.points = o3d.utility.Vector3dVector(bones_pos[0])
# keypoints_center = keypoints.get_center()
# keypoints.points = o3d.utility.Vector3dVector(bones_pos[0])
# skeleton_joints = o3d.geometry.LineSet()
# skeleton_joints.points = o3d.utility.Vector3dVector(bones_pos[0])
# center_skel = skeleton_joints.get_center()
# skeleton_joints.points = o3d.utility.Vector3dVector(bones_pos[0])
# skeleton_joints.lines = o3d.utility.Vector2iVector(body_edges)
# skeleton_joints.colors = o3d.utility.Vector3dVector(colors)

# vis = o3d.visualization.Visualizer()
    
# vis.create_window()
# # This plot the entire skeleton
# vis.add_geometry(skeleton_joints)
# vis.add_geometry(keypoints)

# time.sleep(5)
# for i in range(1000,2003):
#     print(i)
#     new_joints = bones_pos[i]
#     # center_skel = skeleton_joints.get_center()
#     skeleton_joints.points = o3d.utility.Vector3dVector(new_joints)
#     keypoints.points = o3d.utility.Vector3dVector(new_joints)
#     print(new_joints)
#     # This plot the entire skeleton
#     vis.update_geometry(skeleton_joints)
#     vis.update_geometry(keypoints)
    
#     vis.update_renderer()
#     vis.poll_events()

#     time.sleep(0.5)
    
# vis.run()

Found rigid bodies: dict_keys(['Ragnetto'])


In [4]:
def replace_none_with_zero(lst):
    if lst is None:
        return [0, 0, 0]
    else:
        for i in range(len(lst)):
            if lst[i] is None:
                lst[i] = [0, 0, 0]
            elif isinstance(lst[i], list):
                lst[i] = replace_none_with_zero(lst[i])
        return lst

matrix = replace_none_with_zero(bones_pos)

# Extract x, y, z coordinates from the data
x = [point[0] for point in matrix[0]]
y = [point[1] for point in matrix[0]]
z = [point[2] for point in matrix[0]]
positions = np.array([x, y, z]).T

# Create 3D plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

##
###
#####
###
##

# Create the initial point
point = ax.scatter(x[0], y[0], z[0], color='r', s=10)

def update(frame):
     point._offsets3d = (x[:frame], y[:frame], z[:frame])
     
# Set labels
ax.set_xlabel('X')
ax.set_xlim(np.min(x), np.max(x))
ax.set_ylabel('Y')
ax.set_ylim(np.min(y), np.max(y))
ax.set_zlabel('Z')
ax.set_zlim(np.min(z), np.max(z))

ani = animation.FuncAnimation(
    fig, update, frames=len(positions), interval=100)

ani.save("anim.gif", writer="pillow", fps=60)

plt.show()

INFO - 2024-04-16 11:45:11,391 - animation - Animation.save using <class 'matplotlib.animation.PillowWriter'>
